# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
woa_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                woa_evaluation_results['image_name'].append(image_name)
                woa_evaluation_results['thresholds'].append(threshold)
                woa_evaluation_results['fitness_function'].append(fitness_function)
                woa_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # WOA optimization
                    woa = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    whales, best_thresholds = woa.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(woa.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(woa.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                woa_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                woa_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                woa_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                woa_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                woa_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                woa_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                woa_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                woa_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                woa_evaluation_results['Regions'].append(mean_regions)
                woa_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                woa_evaluation_results['Fitness'].append(fitness_values)
                woa_evaluation_results['MSE'].append(list_mse)
                woa_evaluation_results['RMSE'].append(list_rmse)
                woa_evaluation_results['PSNR'].append(list_psnr)
                woa_evaluation_results['SSIM'].append(list_ssim)
                woa_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7106129145502
1948.7183902321062
1948.7183902321062
1948.6109796339413
1948.6717399702634
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6765950132303
1948.7183902321062
1948.7183902321062
1948.6717399702634
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.6409800416593
1948.6109796339413
1948.6717399702634
Mean Fitness = 1948.7020724978963
	 AirplaneF16.tiff | Threshold =  3
2024.7642699718326
2024.8375098105803
2024.8375098105803
2024.6801094478637
2024.1626775030013
2024.8375098105803
2024.494584235908
2024.8375098105803
2024.8375098105803
2024.7871768318307
2024.6521445628969
2024.805475833105
2024.7502576316608
2023.5973366065514
2024.6657763861986
2024.5430795189563
2024.494584235908
2024

3208.1023398198317
3208.244569298565
Mean Fitness = 3214.547422477495
	 Mandrill.tiff | Threshold =  2
1548.9952130399997
1548.9859586850848
1548.9952130399997
1548.9952130399997
1548.9952130399997
1548.9952130399997
1549.0271327708897
1549.0271327708897
1548.524495009854
1549.0271327708897
1548.9952130399997
1549.0271327708897
1548.9952130399997
1549.0271327708897
1548.9952130399997
1548.9952130399997
1549.0271327708897
1549.0271327708897
1548.9952130399997
1548.9859586850848
1548.9952130399997
1548.9952130399997
1548.9952130399997
1549.0271327708897
1549.0271327708897
1548.9952130399997
1548.9859586850848
1548.9952130399997
1549.0271327708897
1549.0271327708897
Mean Fitness = 1548.9903009048296
	 Mandrill.tiff | Threshold =  3
1639.463360998692
1548.7861468406588
1639.4006244533603
1638.304216775733
1639.413602033513
1639.466171099567
1548.7861468406588
1549.0271327708897
1639.466171099567
1639.338310828638
1548.9952130399997
1639.413602033513
1637.8669704258618
1639.466171099567
154

4213.465239334275
4215.539906564259
4181.777908766622
4180.4740470989
4181.983915226995
4216.233425475646
Mean Fitness = 4210.173338118062


## Visualize Results using DataFrame

In [7]:
woa_evaluation_results_df =pd.DataFrame(woa_evaluation_results)
woa_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[113, 173]",1.893940,1948.702072,2270.689159,47.651720,14.569237,0.794398,0.940215,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.8414766788482666, 1.7659873962402344, 2.454...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2258....","[47.651238369936166, 47.651238369936166, 47.52...","[14.569319775617668, 14.569319775617668, 14.59...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[92, 143, 190]",1.597602,2024.650093,1802.656157,42.456017,15.572361,0.783699,0.952835,"[[92, 255, 255, 190, 190, 190, 255, 190, 255, ...","[1.2421808242797852, 1.793116807937622, 2.1958...","[2024.7642699718326, 2024.8375098105803, 2024....","[1770.1938438415527, 1774.3395538330078, 1774....","[42.07367162301803, 42.12291008267363, 42.1229...","[15.650595347878877, 15.640436268576428, 15.64...","[0.7798626027541222, 0.77934507049225, 0.77934...","[0.9534747458488398, 0.9532349424759257, 0.953..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[80, 125, 169, 202]",1.545825,2068.722363,1158.311625,34.009111,17.504982,0.734364,0.968440,"[[80, 202, 202, 202, 202, 202, 202, 202, 202, ...","[1.5548734664916992, 1.57487154006958, 1.62439...","[2070.0794736113085, 2069.4248542313003, 2068....","[1094.045597076416, 1155.6620635986328, 1042.1...","[33.07636009412789, 33.995029983787816, 32.281...","[17.74044938188586, 17.50249503747468, 17.9516...","[0.7326687134455045, 0.7316245602969808, 0.727...","[0.9693723689797689, 0.9684347893999756, 0.970..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[65, 106, 141, 178, 205]",1.644572,2093.976639,930.440704,30.492083,18.450205,0.744857,0.975164,"[[106, 205, 205, 205, 205, 205, 205, 205, 205,...","[1.4648942947387695, 1.2688310146331787, 1.692...","[2095.856157110527, 2095.310852633459, 2082.66...","[908.9174766540527, 910.3732376098633, 1050.31...","[30.148258268995452, 30.172391976935856, 32.40...","[18.545559067556333, 18.538608786547826, 17.91...","[0.7448772055819616, 0.7447822613545529, 0.738...","[0.9751665573754672, 0.9754772201186336, 0.973..."
4,Lena.png,2,mMasi_entropy,max,"[93, 151]",1.632538,1961.794601,2820.621399,53.109133,13.627483,0.613072,0.883409,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.4819252490997314, 1.527437448501587, 1.5260...","[1961.7943480891424, 1961.8220125153634, 1961....","[2812.934658050537, 2813.567070007324, 2813.56...","[53.03710642607246, 53.04306806744237, 53.0430...","[13.639207168568792, 13.638230885124642, 13.63...","[0.6146728574956875, 0.6121817460412936, 0.612...","[0.8838413432858673, 0.8833090574580458, 0.883..."
5,Lena.png,3,mMasi_entropy,max,"[80, 126, 170]",1.663615,2128.011811,1261.542159,35.517276,17.122236,0.707706,0.934357,"[[170, 170, 170, 170, 170, 170, 170, 170, 170,...","[1.5372343063354492, 1.0915238857269287, 1.843...","[2128.154978585403, 2128.2312088823064, 2125.9...","[1230.5840682983398, 1256.1932487487793, 1311....","[35.079681701782015, 35.44281660292787, 36.220...","[17.229690726536113, 17.140239058541706, 16.95...","[0.706522515277939, 0.7066110166865196, 0.7124...","[0.9342541051208892, 0.9333572280821731, 0.936..."
6,Lena.png,4,mMasi_entropy,max,"[72, 111, 144, 179]",1.886800,2189.046600,812.620237,28.476751,19.048906,0.751628,0.954689,"[[179, 179, 179, 179, 179, 179, 179, 179, 179,...","[1.6819508075714111, 2.3334288597106934, 1.126...","[2191.874720070565, 2189.1390487212266, 2190.4...","[795.6595458984375, 787.6624603271484, 794.273...","[28.2074377762043, 28.065324874783624, 28.1828...","[19.12353083289055, 19.167402132039378, 19.131...","[0.7534048142269026, 0.751966435058748, 0.7530...","[0.9539824804728553, 0.9575074464238671, 0.957..."
7,Lena

# Save results

In [8]:
pickle.dump(woa_evaluation_results_df, open('results/evaluation/WOA/WOA_masi_df.pkl', 'wb'))
pickle.dump(woa_evaluation_results, open('results/evaluation/WOA/WOA_masi_dict.pkl', 'wb'))